In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import shutil
import torch

In [2]:
# Define paths for your dataset
base_path = 'C:\\Users\\nandu\\Downloads\\Robotics-MFC-S4-D12\\pothole_image_data'
normal_path = os.path.join(base_path, 'normal')
potholes_path = os.path.join(base_path, 'potholes')

# Create train and val directories under pothole_image_data
train_images_path = os.path.join(base_path, 'train', 'images')
train_labels_path = os.path.join(base_path, 'train', 'labels')
val_images_path = os.path.join(base_path, 'val', 'images')
val_labels_path = os.path.join(base_path, 'val', 'labels')

# Create directories if they don't exist
for path in [train_images_path, train_labels_path, val_images_path, val_labels_path]:
    os.makedirs(path, exist_ok=True)

# Function to split dataset into train and val
def split_dataset(normal_path, potholes_path, train_ratio=0.8):
    # Get all normal and pothole image paths
    normal_images = [os.path.join(normal_path, f) for f in os.listdir(normal_path) if f.endswith('.jpg')]
    pothole_images = [os.path.join(potholes_path, f) for f in os.listdir(potholes_path) if f.endswith('.jpg')]
    
    # Split normal images
    normal_train, normal_val = train_test_split(normal_images, train_size=train_ratio, random_state=42)
    # Split pothole images
    pothole_train, pothole_val = train_test_split(pothole_images, train_size=train_ratio, random_state=42)
    
    # Copy images to train and val directories
    for img_path in normal_train:
        shutil.copy(img_path, os.path.join(train_images_path, os.path.basename(img_path)))
        # Create empty label file for normal (no potholes)
        label_path = os.path.join(train_labels_path, os.path.splitext(os.path.basename(img_path))[0] + '.txt')
        with open(label_path, 'w') as f:
            pass  # Empty file indicates no potholes
    
    for img_path in normal_val:
        shutil.copy(img_path, os.path.join(val_images_path, os.path.basename(img_path)))
        label_path = os.path.join(val_labels_path, os.path.splitext(os.path.basename(img_path))[0] + '.txt')
        with open(label_path, 'w') as f:
            pass  # Empty file indicates no potholes
    
    for img_path in pothole_train:
        shutil.copy(img_path, os.path.join(train_images_path, os.path.basename(img_path)))
        # You need to provide or generate YOLO annotations for potholes
        label_path = os.path.join(train_labels_path, os.path.splitext(os.path.basename(img_path))[0] + '.txt')
        # Placeholder for pothole annotation (replace with actual annotations)
        # Example: '0 0.5 0.5 0.2 0.2' for a pothole centered with width/height 20% of image
        # You must create these manually or using a tool like LabelImg/Roboflow
        with open(label_path, 'w') as f:
            f.write('0 0.5 0.5 0.2 0.2')  # Example, replace with real annotations
    
    for img_path in pothole_val:
        shutil.copy(img_path, os.path.join(val_images_path, os.path.basename(img_path)))
        label_path = os.path.join(val_labels_path, os.path.splitext(os.path.basename(img_path))[0] + '.txt')
        with open(label_path, 'w') as f:
            f.write('0 0.5 0.5 0.2 0.2')  # Example, replace with real annotations

# Split the dataset
split_dataset(normal_path, potholes_path)

# Create dataset.yaml with absolute paths
dataset_yaml = '''
train: {train_images_path}
val: {val_images_path}
nc: 1  # Number of classes (1 for potholes)
names: ['pothole']  # Class names
'''

# Write dataset.yaml to file with absolute paths
with open('dataset.yaml', 'w') as f:
    f.write(dataset_yaml.format(train_images_path=train_images_path.replace('\\', '/'), 
                               val_images_path=val_images_path.replace('\\', '/')))

# Update Ultralytics settings to use the correct dataset directory (optional)
settings_path = os.path.join(os.path.expanduser('~'), 'AppData', 'Roaming', 'Ultralytics', 'settings.json')
if os.path.exists(settings_path):
    import json
    with open(settings_path, 'r') as f:
        settings = json.load(f)
    settings['datasets_dir'] = base_path  # Update to your pothole_image_data directory
    with open(settings_path, 'w') as f:
        json.dump(settings, f, indent=4)
else:
    print(f"Settings file not found at {settings_path}. Using default dataset directory.")

# Load or train a custom YOLOv8 model
model = YOLO('yolov8n.pt')  # Start with pre-trained YOLOv8 nano model

# Train the model on your pothole dataset
model.train(
    data='dataset.yaml',
    epochs=100,  # Increase for better accuracy, adjust as needed
    imgsz=640,   # Image size, adjust based on your images (YOLOv8 default is 640)
    batch=16,    # Batch size, adjust based on your GPU/CPU memory
    project='runs/train',  # Output directory for training results
    name='pothole_detection',  # Experiment name
    exist_ok=True  # Allow overwriting existing runs
)

# Load the best trained model
best_model = YOLO('runs/train/pothole_detection/weights/best.pt')  # Path to best model weights

Ultralytics 8.3.82  Python-3.12.1 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i7-13700H)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/train, name=pothole_detection, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

train: Scanning C:\Users\nandu\Downloads\Robotics-MFC-S4-D12\pothole_image_data\train\labels... 324 images, 61 backgrounds, 0 corrupt: 100%|██████████| 324/324 [00:00<00:00, 595.49it/s]

train: WARNING  C:\Users\nandu\Downloads\Robotics-MFC-S4-D12\pothole_image_data\train\images\253.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\nandu\Downloads\Robotics-MFC-S4-D12\pothole_image_data\train\images\266.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Users\nandu\Downloads\Robotics-MFC-S4-D12\pothole_image_data\train\images\279.jpg: corrupt JPEG restored and saved
train: New cache created: C:\Users\nandu\Downloads\Robotics-MFC-S4-D12\pothole_image_data\train\labels.cache



val: Scanning C:\Users\nandu\Downloads\Robotics-MFC-S4-D12\pothole_image_data\val\labels.cache... 110 images, 44 backgrounds, 0 corrupt: 100%|██████████| 110/110 [00:00<?, ?it/s]


Plotting labels to runs\train\pothole_detection\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\train\pothole_detection
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100         0G     0.7157     0.8222       1.37          4        640: 100%|██████████| 21/21 [00:54<00:00,  2.61s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.40s/it]

                   all        110         66      0.858      0.919      0.921      0.809



100 epochs completed in 1.773 hours.
Optimizer stripped from runs\train\pothole_detection\weights\last.pt, 6.3MB
Optimizer stripped from runs\train\pothole_detection\weights\best.pt, 6.3MB

Validating runs\train\pothole_detection\weights\best.pt...
Ultralytics 8.3.82  Python-3.12.1 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i7-13700H)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.28s/it]


                   all        110         66      0.874      0.909       0.94      0.815
Speed: 0.7ms preprocess, 33.8ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\train\pothole_detection


In [7]:
def preprocess_image_with_edges(frame):
    # Convert to grayscale if frame is color
    if len(frame.shape) == 3:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        gray = frame
    
    # Apply bilateral filtering for noise reduction and edge preservation
    smoothed = cv2.bilateralFilter(gray, 9, 75, 75)  # d=9, sigmaColor=75, sigmaSpace=75
    
    # Enhance contrast with CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(smoothed)
    
    # Use Canny edge detection with adaptive thresholding
    edges = cv2.Canny(enhanced, 50, 150)  # Adjust thresholds as needed
    
    # Morphological operations to clean edges
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)
    edges = cv2.erode(edges, kernel, iterations=1)
    
    return frame, edges  # Return original frame and edges for consistency

def get_grid_position(width, height, center_x, center_y, grid_size=10):
    # Divide the image into a grid_size x grid_size grid
    grid_width = width // grid_size
    grid_height = height // grid_size
    
    # Calculate grid cell (row, column) based on center coordinates
    row = center_y // grid_height
    col = center_x // grid_width
    
    # Ensure row and col are within bounds
    row = min(max(0, row), grid_size - 1)
    col = min(max(0, col), grid_size - 1)
    
    return row, col

def detect_pothole_in_image(model, image_path, scale_factor=0.1, grid_size=10):
    # Preprocess the image
    original_image, edges = preprocess_image_with_edges(cv2.imread(image_path))
    
    # Load and predict with YOLOv8
    results = model(image_path, conf=0.5)  # Confidence threshold of 0.5
    
    # Convert BGR to RGB for display
    rgb_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    
    pothole_detected = False
    image_width, image_height = rgb_image.shape[1], rgb_image.shape[0]
    
    for result in results:
        boxes = result.boxes  # Get bounding boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)  # Convert to CPU and int
            conf = box.conf.cpu().numpy()[0]  # Confidence score
            
            if conf >= 0.5:  # Ensure confidence threshold
                pothole_detected = True
                print(f"Pothole Detected with confidence {conf:.2f}!")
                
                # Calculate width and height in pixels
                w = x2 - x1
                h = y2 - y1
                
                # Calculate dimensions in cm (adjust scale_factor based on image resolution and real-world distance)
                width_cm = w * scale_factor
                height_cm = h * scale_factor
                
                # Calculate position (center coordinates in pixels)
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                
                # Get grid position (row, column) in a 10x10 grid
                grid_row, grid_col = get_grid_position(image_width, image_height, center_x, center_y, grid_size)
                
                # Draw bounding box and text
                cv2.rectangle(rgb_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                text = f'Pothole: {width_cm:.2f}cm x {height_cm:.2f}cm\nGrid Position: Row {grid_row}, Col {grid_col}'
                cv2.putText(rgb_image, text, (x1, y1-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    if not pothole_detected:
        print("No Pothole Detected.")
        cv2.putText(rgb_image, 'No Pothole', (50, 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Display the result
    plt.figure(figsize=(10, 8))
    plt.imshow(rgb_image)
    plt.title('Pothole Detection Result')
    plt.axis('off')
    plt.show()

    # Optionally save the annotated image
    output_path = 'pothole_detection_result.jpg'
    cv2.imwrite(output_path, cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR))
    print(f"Annotated image saved as {output_path}")

In [8]:
def detect_potholes_in_video(model, video_path, output_path='pothole_detection_output.avi', scale_factor=0.1, grid_size=10):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file at {video_path}")
        return
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define the codec and create VideoWriter object to save the output video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can use 'MJPG' or other codecs
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video
        
        frame_count += 1
        print(f"Processing frame {frame_count}")
        
        # Preprocess the frame
        original_frame, edges = preprocess_image_with_edges(frame)
        # No need to reshape for YOLOv8, as it handles raw frames directly
        
        # Predict if a pothole is present
        results = model(frame, conf=0.5)  # YOLOv8 prediction on the raw frame (BGR)
        
        # Convert BGR frame to RGB for display and processing
        rgb_frame = cv2.cvtColor(original_frame, cv2.COLOR_BGR2RGB)
        
        pothole_detected = False
        
        for result in results:
            boxes = result.boxes  # Get bounding boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                conf = box.conf.cpu().numpy()[0]
                
                if conf >= 0.5:
                    pothole_detected = True
                    print(f"Frame {frame_count}: Pothole Detected with confidence {conf:.2f}!")
                    
                    # Calculate width and height in pixels
                    w = x2 - x1
                    h = y2 - y1
                    
                    # Calculate dimensions in cm
                    width_cm = w * scale_factor
                    height_cm = h * scale_factor
                    
                    # Calculate position (center coordinates in pixels)
                    center_x = (x1 + x2) // 2
                    center_y = (y1 + y2) // 2
                    
                    # Get grid position (row, column) in a 10x10 grid
                    grid_row, grid_col = get_grid_position(frame_width, frame_height, center_x, center_y, grid_size)
                    
                    # Draw bounding box and text
                    cv2.rectangle(rgb_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    text = f'Pothole: {width_cm:.2f}cm x {height_cm:.2f}cm\nGrid Position: Row {grid_row}, Col {grid_col}'
                    cv2.putText(rgb_frame, text, (x1, y1-10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        if not pothole_detected:
            cv2.putText(rgb_frame, 'No Pothole', (50, 50),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        # Convert back to BGR for OpenCV output and writing to video
        annotated_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
        
        # Write the annotated frame to the output video
        out.write(annotated_frame)
    
    # Release resources
    cap.release()
    out.release()
    print(f"Annotated video saved as {output_path}")

In [9]:
# Process a single image
image_path = 'pothole.jpg'  
detect_pothole_in_image(best_model, image_path, scale_factor=0.1, grid_size=10)


image 1/1 c:\Users\nandu\Downloads\Robotics-MFC-S4-D12\pothole.jpg: 416x640 (no detections), 55.2ms
Speed: 1.8ms preprocess, 55.2ms inference, 0.3ms postprocess per image at shape (1, 3, 416, 640)
No Pothole Detected.


<Figure size 1000x800 with 1 Axes>

Annotated image saved as pothole_detection_result.jpg


In [10]:
# Process a video
video_path = 'demo.mp4'  
detect_potholes_in_video(best_model, video_path, scale_factor=0.1, grid_size=10)

Processing frame 1

0: 384x640 (no detections), 52.7ms
Speed: 1.4ms preprocess, 52.7ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 2

0: 384x640 (no detections), 40.1ms
Speed: 1.4ms preprocess, 40.1ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 3

0: 384x640 (no detections), 41.5ms
Speed: 1.2ms preprocess, 41.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 4

0: 384x640 (no detections), 44.3ms
Speed: 1.3ms preprocess, 44.3ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 5

0: 384x640 (no detections), 47.5ms
Speed: 1.7ms preprocess, 47.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 6

0: 384x640 (no detections), 43.2ms
Speed: 1.7ms preprocess, 43.2ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
Processing frame 7

0: 384x640 (no detections), 41.0ms
Speed: 1.8ms preprocess, 41.0ms i